In [4]:
import gym
import gym.wrappers
import tianshou as ts
import torch
import numpy as np
import platoonenv
from pprint import pprint

In [5]:
# envName = "CartPole-v1"
envName = "TorchPlatoon-v0"
params=platoonenv.PlatoonEnvParams(
    num_vehicles=10,
    num_vulns=4,
    num_attack=1,
    attack_interval=2
)
def make_env():
    env = gym.make(envName, render_mode="human", params=params)
    env = gym.wrappers.TimeLimit(env, max_episode_steps=20)
    return env
env = make_env()

d:\Repos\School\Vehicular-Game-Theory\platoongame\platoonenv.py:40: UserWarning: WARN: pls ignore gym complaints about observation space
  gym.logger.warn("pls ignore gym complaints about observation space")
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [3]:
env.reset()
for i in range(1000):
    env.render()
    obs, rew, done, trunc, info = env.step(env.action_space.sample())
    if trunc or done:
        print(f"done after {i} steps")
        break
print(i)

KeyboardInterrupt: 

In [8]:
train_envs = ts.env.DummyVectorEnv([lambda: make_env() for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: make_env() for _ in range(100)])

d:\Repos\School\Vehicular-Game-Theory\platoongame\platoonenv.py:40: UserWarning: WARN: pls ignore gym complaints about observation space
  gym.logger.warn("pls ignore gym complaints about observation space")
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [11]:
from tianshou.utils.net.common import Net
obs_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(obs_shape, action_shape, hidden_sizes=(128,128,128,128))
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [12]:
policy = ts.policy.DQNPolicy(net, optim, discount_factor=0.9, estimation_step=3, target_update_freq=320)

In [13]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, train_envs, exploration_noise=True)

In [14]:
print(env.spec)
print("Threshold:", env.spec.reward_threshold)

EnvSpec(id='TorchPlatoon-v0', entry_point='platoonenv:PlatoonEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=100, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'human', 'params': PlatoonEnvParams(num_vehicles=3, num_vulns=2, num_attack=1, attack_interval=2)}, namespace=None, name='TorchPlatoon', version=0)
Threshold: None


In [15]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=20, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= 400,
    # stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
)
print(f"Finished training! Use {result['duration']}")

Epoch #1: 10001it [00:09, 1088.01it/s, env_step=10000, len=100, loss=4.648, n/ep=10, n/st=10, rew=69.70]                           


Epoch #1: test_reward: 87.270000 ± 90.125896, best_reward: 87.270000 ± 90.125896 in #1


Epoch #2: 10001it [00:08, 1187.97it/s, env_step=20000, len=100, loss=4.182, n/ep=10, n/st=10, rew=67.80]                           


Epoch #2: test_reward: 131.810000 ± 85.272703, best_reward: 131.810000 ± 85.272703 in #2


Epoch #3: 10001it [00:08, 1197.44it/s, env_step=30000, len=100, loss=3.098, n/ep=10, n/st=10, rew=98.60]                           


...

Epoch #18: test_reward: 136.550000 ± 95.983892, best_reward: 161.040000 ± 75.161016 in #4


Epoch #19: 10001it [00:07, 1276.94it/s, env_step=190000, len=100, loss=1.899, n/ep=10, n/st=10, rew=185.70]                           


Epoch #19: test_reward: 157.410000 ± 72.837778, best_reward: 161.040000 ± 75.161016 in #4


Epoch #20: 10001it [00:08, 1195.78it/s, env_step=200000, len=100, loss=1.856, n/ep=10, n/st=10, rew=134.60]                           


Epoch #20: test_reward: 146.720000 ± 88.285229, best_reward: 161.040000 ± 75.161016 in #4
Finished training! Use 223.64s


In [16]:
result

{'duration': '223.64s',
 'train_time/model': '105.09s',
 'test_step': 210000,
 'test_episode': 2100,
 'test_time': '54.59s',
 'test_speed': '3847.21 step/s',
 'best_reward': 161.04,
 'best_result': '161.04 ± 75.16',
 'train_step': 200000,
 'train_episode': 2000,
 'train_time/collector': '63.96s',
 'train_speed': '1183.07 step/s'}

In [18]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, make_env(), exploration_noise=False)
collector.collect(n_episode=10, render=1 / 60)

d:\Repos\School\Vehicular-Game-Theory\platoongame\platoonenv.py:40: UserWarning: WARN: pls ignore gym complaints about observation space
  gym.logger.warn("pls ignore gym complaints about observation space")
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\gym\utils\passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\tianshou\data\collector.py:68: UserWarning: S

KeyboardInterrupt: 